### Processing a demo fastq file using biopython 

### Read FASTQ file

In [2]:
from collections import defaultdict
import gzip
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

from Bio import SeqIO

In [11]:
recs = SeqIO.parse(open(
    '/Users/stevennguyen/Bioinformatics/demo.fastq', 'rt',encoding='utf-8'), 'fastq')
rec = next(recs)
print(rec)
print(rec.letter_annotations['phred_quality'])

ID: SRR960459.1
Name: SRR960459.1
Description: SRR960459.1 HWI-ST330:304:H045HADXX:1:1101:1162:2055 length=100
Number of features: 0
Per letter annotation for: phred_quality
Seq('NAGAACTTGGCGGCGAATGGGCTGACCGCTTCCTCGTGCTTTACGGTATCGCCG...GTT')
[2, 16, 28, 35, 35, 37, 37, 37, 39, 39, 39, 38, 39, 40, 41, 41, 41, 41, 40, 41, 41, 41, 38, 36, 38, 38, 32, 37, 38, 33, 39, 39, 36, 39, 38, 37, 33, 37, 37, 35, 36, 35, 36, 34, 35, 35, 32, 28, 28, 34, 33, 31, 33, 35, 35, 35, 35, 35, 30, 26, 33, 12, 27, 34, 33, 35, 35, 35, 29, 33, 33, 33, 33, 35, 35, 33, 20, 27, 31, 35, 35, 35, 34, 35, 35, 33, 31, 12, 24, 32, 34, 35, 35, 35, 35, 33, 30, 33, 27, 30]


### Distribution of nucleotides for each read

In [15]:
from Bio.SeqUtils import gc_fraction

gc_data = ((gc_fraction(rec.seq), len(rec.seq)) for rec in SeqIO.parse(open('/Users/stevennguyen/Bioinformatics/demo.fastq', 'rt',encoding='utf-8'), 'fastq'))

total_gc,total_bases = map(sum,zip(*gc_data))

print(f'Total GC content: {total_gc/total_bases:.2%}')

Total GC content: 0.48%


### Assessing quality of reads 
